<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [139]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [145]:
df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
data=np.array([df.iloc[0:5][5],df.iloc[800000:800005][5]])

print(data)

[["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
  "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
  '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
  'my whole body feels itchy and like its on fire '
  "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "]
 ['I LOVE @Health4UandPets u guys r the best!! '
  'im meeting up with one of my besties tonight! Cant wait!!  - GIRL TALK!!'
  '@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart. '
  'Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup'
  '@LovesBrooklyn2 he has that effect on everyone ']]


# Traitement

In [146]:
def processing(data):

  #MINUSCULES
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=data[i][j].lower()
  print(data)

  #TOKENIZATION
  tokenizer = nltk.RegexpTokenizer(r'\w+')

  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=tokenizer.tokenize(data[i][j])
  print(data)

  #STOPWORDS
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
      
  print(data)

  #STEMMING
  stemmer=LancasterStemmer()
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[stemmer.stem(w) for w in data[i][j]]
  print(data)

  #LEMMATIZATION
  Word_Lemmatizer = WordNetLemmatizer()
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]
  print(data)

  return data

In [147]:
datas=processing(data)

[["@switchfoot http://twitpic.com/2y1zl - awww, that's a bummer.  you shoulda got david carr of third day to do it. ;d"
  "is upset that he can't update his facebook by texting it... and might cry as a result  school today also. blah!"
  '@kenichan i dived many times for the ball. managed to save 50%  the rest go out of bounds'
  'my whole body feels itchy and like its on fire '
  "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there. "]
 ['i love @health4uandpets u guys r the best!! '
  'im meeting up with one of my besties tonight! cant wait!!  - girl talk!!'
  '@darealsunisakim thanks for the twitter add, sunisa! i got to meet you once at a hin show here in the dc area and you were a sweetheart. '
  'being sick can be really cheap when it hurts too much to eat real food  plus, your friends make you soup'
  '@lovesbrooklyn2 he has that effect on everyone ']]
[[list(['switchfoot', 'http', 'twitpic', 'com', '2y1zl', 'awww', 'that

In [153]:
def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y" with zeros (≈2 lines)
    a = np.zeros((n_a,m,T_x))
    y_pred = np.zeros((n_y,m,T_x))
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache (≈1 line)
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches" (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [154]:
np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)


NameError: ignored

In [148]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(5,2))
model.compile('rmsprop', 'mse')